In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
from collections import Counter
from nltk.tokenize import word_tokenize
import torch

## Load Model

In [35]:
model_path = "./gpt2_model_trained_1"
model_1 = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_1 = AutoTokenizer.from_pretrained(model_path)

In [36]:
model_path = "./gpt2_model_trained_2"
model_2 = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_2 = AutoTokenizer.from_pretrained(model_path)

In [37]:
model_path = "./gpt2_model_trained_3"
model_3 = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer_3 = AutoTokenizer.from_pretrained(model_path)

## Prompt

In [54]:
prompt = """
En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
"""

input_ids_1 = tokenizer_1(prompt, return_tensors="pt").input_ids

output = model_1.generate(
    input_ids=input_ids_1,
    max_length=300,
    min_length=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.8,
    do_sample=True,
    pad_token_id=tokenizer_1.eos_token_id,
)

generated_text_1 = tokenizer_1.decode(output[0], skip_special_tokens=True)
print(generated_text_1)


En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
Una noche tan tranquila mas hermosa voy encontrarme amor llegar tarde aquel mismo mañana temprano llegaste tierra luna contigo quiero ir pues va ser solo sueño vida pronto va tenerte asi cuerpo alma volver aunque nunca vuelva volver dar gracias ay cielo ahora adios siempre veo you might also likecuando llegue mañana vuelvo regresar vas vere camino vez ayer conocide extraño mundo junto hoy cerca lado esperando pueda ver si miras ojos luz piel niña ven vi venir lejos dejas pensar oh why digandose sera lugar esperare buscarle uoh yeaah amor llegara amanecer busque puedo hacer si si ves capaz conmigo amor ir caminando rumbo esperar sabes bien sabra quieres encontrar toda camino siento cada detalle forma diferente da cuenta tiempo igual pasado soledad ah hara dios ay noche tanta gente feliz vivir juntos hace falta calor agua aguanto naci buscando lluvia mar estrellas salen flores rosa rosas d

In [55]:
prompt = """
En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
"""

input_ids_2 = tokenizer_2(prompt, return_tensors="pt").input_ids

output = model_2.generate(
    input_ids=input_ids_2,
    max_length=300,
    min_length=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.8,
    do_sample=True,
    pad_token_id=tokenizer_2.eos_token_id,
)

generated_text_2 = tokenizer_2.decode(output[0], skip_special_tokens=True)
print(generated_text_2)


En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
lores siempre va igual vida entera vanidad tiempo mas dias voy conocerme si miro hacer preguntas ¿que puedo hacer cosas tan solo dos amigos saben hablar nunca habra sido parte de nadie vivo ahora sabe sentirte bien siento asi estare esperando vine aqui amor sabes decir mentiras oh digo verdad piensas quiero mas ser feliz vivir aunque tener tantas ganas quererlo puedes creer veces conmigo quise ir lejos you might also likeyo ahora seguro saberes amo amo ahora mismo sabras quiero aunque dices duele mundo contigo extraño importa lado vuelvas lado cada vez sigo haciendo aquello debia ser libre aceptar perdonar corazon sera culpable gente vengo pensar dejare olvidar razon alguien cree culpa da pena vale sufrir hace falta pa perder dinero culpa vale sufrir hecho mejor vale tener valor mejor vale tener merezco oir pagar woh ow okay ozuy osamo otobo ire soñarte primera vez mas ves mal dire cariñ

In [45]:
prompt = """
En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
"""

input_ids_3 = tokenizer_3(prompt, return_tensors="pt").input_ids

output = model_3.generate(
    input_ids=input_ids_3,
    max_length=300,
    min_length=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.8,
    do_sample=True,
    pad_token_id=tokenizer_3.eos_token_id,
)

generated_text_3 = tokenizer_3.decode(output[0], skip_special_tokens=True)
print(generated_text_3)


En México me encuentro, el sol brilla al pasar,  
colores en las calles, mil historias al andar.  
 Suspirar conmigo camino caminar siempre lado ciudad voy contando vida mia quiero darte mano dio mano quiero darme beso siempre quiero darle beso nunca imagine quisiera tener vayas solo amor dijiste si lado aqui junto quiero darlo beso siempre quiero darles beso besos si dos dias enamorado mujer see luis miguel liveget tickets as low as you might also like


## Evaluation

In [46]:
def calculate_perplexity(model, input_ids):
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

In [47]:
def distinct_n(text, n):
    tokens = text.split()
    ngrams = zip(*[tokens[i:] for i in range(n)])
    unique_ngrams = set(ngrams)
    return len(unique_ngrams) / len(tokens) if tokens else 0

In [48]:
def lexical_diversity(text):
    tokens = word_tokenize(text)
    unique_tokens = set(tokens)
    return len(unique_tokens) / len(tokens) if tokens else 0

In [49]:
# Load a pre-trained embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def local_coherence(text):
    sentences = text.split('.')
    coherence_scores = []
    for i in range(len(sentences) - 1):
        emb1 = embedding_model.encode(sentences[i], convert_to_tensor=True)
        emb2 = embedding_model.encode(sentences[i + 1], convert_to_tensor=True)
        similarity = util.cos_sim(emb1, emb2).item()
        coherence_scores.append(similarity)
    return sum(coherence_scores) / len(coherence_scores) if coherence_scores else 0

In [50]:
def evaluate_text(model, input_ids, text):
    return {
        "Perplexity": calculate_perplexity(model, input_ids),
        "Distinct-1": distinct_n(text, 1),
        "Distinct-2": distinct_n(text, 2),
        "Lexical Diversity": lexical_diversity(text),
        "Local Coherence": local_coherence(text),
    }

In [56]:
metrics_1 = evaluate_text(model_1, input_ids_1,generated_text_1)

In [57]:
metrics_2 = evaluate_text(model_2, input_ids_2,generated_text_2)

In [58]:
metrics_3 = evaluate_text(model_3, input_ids_3,generated_text_3)

In [59]:
metrics_1

{'Perplexity': 354.3334045410156,
 'Distinct-1': 0.8600823045267489,
 'Distinct-2': 0.9423868312757202,
 'Lexical Diversity': 0.854251012145749,
 'Local Coherence': 0.3659774661064148}

In [60]:
metrics_2

{'Perplexity': 611.615234375,
 'Distinct-1': 0.9018691588785047,
 'Distinct-2': 0.9813084112149533,
 'Lexical Diversity': 0.8944954128440367,
 'Local Coherence': 0.4278242290019989}

In [61]:
metrics_3

{'Perplexity': 456.8043518066406,
 'Distinct-1': 0.8181818181818182,
 'Distinct-2': 0.961038961038961,
 'Lexical Diversity': 0.8024691358024691,
 'Local Coherence': 0.4308878183364868}